In [16]:
import pandas as pd
import requests
from envs import env
from jsonpath_rw import jsonpath, parse
from dataclasses import dataclass

In [24]:
DATA_GOV_API_KEY=env("DATA_GOV_API_KEY")
DATA_GOV_API_KEY="kxt1UdiT7CfYrNyBcmn0eWiUms3MyUCmBmBg6KKU"

In [25]:
emissions_df = pd.read_csv('./data/GHG-emissions-by-life-cycle-stage-OurWorldinData-upload.csv')
emissions_df['Food product'].values

array(['Wheat & Rye (Bread)', 'Maize (Meal)', 'Barley (Beer)', 'Oatmeal',
       'Rice', 'Potatoes', 'Cassava', 'Cane Sugar', 'Beet Sugar',
       'Other Pulses', 'Peas', 'Nuts', 'Groundnuts', 'Soymilk', 'Tofu',
       'Soybean Oil', 'Palm Oil', 'Sunflower Oil', 'Rapeseed Oil',
       'Olive Oil', 'Tomatoes', 'Onions & Leeks', 'Root Vegetables',
       'Brassicas', 'Other Vegetables', 'Citrus Fruit', 'Bananas',
       'Apples', 'Berries & Grapes', 'Wine', 'Other Fruit', 'Coffee',
       'Dark Chocolate', 'Beef (beef herd)', 'Beef (dairy herd)',
       'Lamb & Mutton', 'Pig Meat', 'Poultry Meat', 'Milk', 'Cheese',
       'Eggs', 'Fish (farmed)', 'Shrimps (farmed)'], dtype=object)

In [26]:
# create mapping of GHG food groups to FDC IDs
@dataclass
class GHG_FDC_MAP:
    ghg_product_name: str
    fdc_id: int

GHG_FDC_MAPPING = [
    GHG_FDC_MAP(
        ghg_product_name='Wheat & Rye (Bread)', 
        fdc_id=746760,
    ),
    # 'Maize (Meal)', 'Barley (Beer)', 'Oatmeal',
    #    'Rice', 'Potatoes', 'Cassava', 'Cane Sugar', 'Beet Sugar',
    #    'Other Pulses', 'Peas', 'Nuts', 'Groundnuts', 'Soymilk', 'Tofu',
    #    'Soybean Oil', 'Palm Oil', 'Sunflower Oil', 'Rapeseed Oil',
    #    'Olive Oil', 'Tomatoes', 'Onions & Leeks', 'Root Vegetables',
    #    'Brassicas', 'Other Vegetables', 'Citrus Fruit', 'Bananas',
    #    'Apples', 'Berries & Grapes', 'Wine', 'Other Fruit', 'Coffee',
    #    'Dark Chocolate', 'Beef (beef herd)', 'Beef (dairy herd)',
    #    'Lamb & Mutton', 'Pig Meat', 'Poultry Meat', 'Milk', 'Cheese',
    #    'Eggs', 'Fish (farmed)', 'Shrimps (farmed)'


]

In [27]:
fdc_details = []
for food in GHG_FDC_MAPPING:
    fdc_detail_response = requests.get(f"https://api.nal.usda.gov/fdc/v1/{food.fdc_id}?api_key={DATA_GOV_API_KEY}")
    fdc_detail = fdc_detail_response.json()
    fdc_details.append(fdc_detail)
    

In [28]:
fdc_details

'Fatty acids, total trans-dienoic',
     'rank': 15601,
     'unitName': 'g'},
    'foodNutrientDerivation': {'id': 4,
     'code': 'AS',
     'description': 'Summed',
     'foodNutrientSource': {'id': 1,
      'code': '1',
      'description': 'Analytical or derived from analytical'}},
    'amount': 0.011},
   {'type': 'FoodNutrient',
    'id': 8514127,
    'nutrient': {'id': 1306,
     'number': '665',
     'name': '18:2 t not further defined',
     'rank': 15610,
     'unitName': 'g'},
    'dataPoints': 2,
    'foodNutrientDerivation': {'id': 1,
     'code': 'A',
     'description': 'Analytical',
     'foodNutrientSource': {'id': 1,
      'code': '1',
      'description': 'Analytical or derived from analytical'}},
    'max': 0.011,
    'min': 0.011,
    'median': 0.011,
    'nutrientAnalysisDetails': [{'subSampleId': 320386,
      'amount': 0.011,
      'nutrientId': 1306,
      'nutrientAcquisitionDetails': [{'sampleUnitId': 320358,
        'purchaseDate': '7/8/2011',
        'stor